### This notebook is for attempting to see if the .pkl existing user recommender model will load and work properly, in order for it to be implemented on the backend of the project.

In [19]:
from sklearn.externals import joblib
import pandas as pd
import numpy as np

### Knn Model

In [5]:
knn = joblib.load('las_vegas_existing_user_model.pkl');

In [7]:
prediction = knn.predict('3nDUQBjKyVor5wV0reJChg', '6fPQJq4f_yiq1NHn0fd11Q');

In [8]:
prediction

Prediction(uid='3nDUQBjKyVor5wV0reJChg', iid='6fPQJq4f_yiq1NHn0fd11Q', r_ui=None, est=3.875, details={'actual_k': 40, 'was_impossible': False})

### SVDpp Model

In [12]:
svdpp = joblib.load('svdpp_las_vegas_existing_user_model.pkl');

#### Experiment with calculating predicted rating of a particular restaurant by a particular user.

In [13]:
predictions_svdpp = svdpp.predict('3nDUQBjKyVor5wV0reJChg', '6fPQJq4f_yiq1NHn0fd11Q')
predictions_svdpp

Prediction(uid='3nDUQBjKyVor5wV0reJChg', iid='6fPQJq4f_yiq1NHn0fd11Q', r_ui=None, est=3.675397133005485, details={'was_impossible': False})

In [17]:
us_restaurant_review_lasvegas_nv = pd.read_csv("yelp_dataset/us_restaurant_review_lasvegas_nv.csv")

#### Grab business_ids that user_id 3nDUQBjKyVor5wV0reJChg did not rate in order to recommend restaurants they have not visited

In [20]:
business_ids = us_restaurant_review_lasvegas_nv['business_id'].unique()
business_ids_rated_by_user = us_restaurant_review_lasvegas_nv.loc[us_restaurant_review_lasvegas_nv['user_id'] == '3nDUQBjKyVor5wV0reJChg', 'business_id']
business_ids_to_predict = np.setdiff1d(business_ids, business_ids_rated_by_user)

#### Predict the review_ratings for each business_id that user_id 3nDUQBjKyVor5wV0reJChg did not rate

In [21]:
testset = [['3nDUQBjKyVor5wV0reJChg', business_id, 4.] for business_id in business_ids_to_predict]
predictions_testset = svdpp.test(testset)
predictions_testset

[Prediction(uid='3nDUQBjKyVor5wV0reJChg', iid='--9e1ONYQuAa-CB_Rrw7Tw', r_ui=4.0, est=4.4003964093669845, details={'was_impossible': False}),
 Prediction(uid='3nDUQBjKyVor5wV0reJChg', iid='-0RkJ_uIduNLWQrphbADRw', r_ui=4.0, est=3.0744625677093667, details={'was_impossible': False}),
 Prediction(uid='3nDUQBjKyVor5wV0reJChg', iid='-1m9o3vGRA8IBPNvNqKLmA', r_ui=4.0, est=4.228651451891179, details={'was_impossible': False}),
 Prediction(uid='3nDUQBjKyVor5wV0reJChg', iid='-1vfRrlnNnNJ5boOVghMPA', r_ui=4.0, est=2.7197282282102377, details={'was_impossible': False}),
 Prediction(uid='3nDUQBjKyVor5wV0reJChg', iid='-3zffZUHoY8bQjGfPSoBKQ', r_ui=4.0, est=3.6530172353081296, details={'was_impossible': False}),
 Prediction(uid='3nDUQBjKyVor5wV0reJChg', iid='-8R_-EkGpUhBk55K9Dd4mg', r_ui=4.0, est=2.8056861035488243, details={'was_impossible': False}),
 Prediction(uid='3nDUQBjKyVor5wV0reJChg', iid='-9YyInW1wapzdNZrhQJ9dg', r_ui=4.0, est=2.6021517524716495, details={'was_impossible': False}),
 Predic

#### Output the top 30 business_ids with the top 30 best predicted review_rating

In [22]:
predicted_ratings = np.array([pred.est for pred in predictions_testset])
index_max_pred_rating = np.argpartition(predicted_ratings,-30)[-30:]
business_id_recommended = business_ids_to_predict[index_max_pred_rating]
print('Top 30 businesses:\n {0}, \npredicted review_ratings: \n{1}'.format(business_id_recommended, predicted_ratings[index_max_pred_rating]))

Top 10 businesses:
 ['RwMLuOkImBIqqYj4SSKSPg' '8GOle3PqqZnni4U-vXBKQA'
 'AMTNJbYbu0OMMAkx4meQxw' 'YTCCJ3ShO-zg0dlx1nk6dw'
 'OETh78qcgDltvHULowwhJg' 'LpGCzgAKNSpzDNEUMVVtZA'
 'SUd3IyEN7n5OYEGyt_U5lg' 'HykZIiaHS35N2c5_QHoClA'
 'hV4j36jlyeBwTW_9uQiTjQ' 'hPclwunHi36YV8cPghWJBg'
 'IFvBnwcWJ54BgMzVnCxIrQ' 'ZibmYdOPKLlqDM9oR6xzOA'
 'I6EDDi4-Eq_XlFghcDCUhw' 'dPxZI9lrKTl5dvFfnb1_Ig'
 'YRyYbOSwvHkZsZOLv98oQg' 'eaNenRk_liZBERFFLCXqqQ'
 'hHe1mFDxxrtejXHfiysJJg' 'D6K3VT6S8FwFm4u5uN8T8g'
 'CWNMLT-ppaUjLMmrnYDPVg' 'lTixk1IT-D4hoMZ6LgoyHQ'
 'mUdalYTuAtnZm2K9zoH27Q' '698nukvxe7mq2Fra5vDikQ'
 'UMqfeCItzQ2glr4d9apGlA' 'PXShA3JZMXr2mEH3on5clw'
 '3QQdF1RT43sVq2YxeE_iHw' '2hSIeCX2cV-chFcBBXrZZA'
 'vT812nRctS62PVxUverayw' '0YG_kM5caZhtwd6nbnei6A'
 'RVt3dKDevhEs0gVMYeL6_w' 'xxAGaqYnboeR0SilS7DbAg'], 
predicted review_ratings: 
[4.58656019 4.58809837 4.6027465  4.60843378 4.60730508 4.61155124
 4.62463478 4.62044461 4.62452631 4.63057888 4.70969844 4.80114199
 4.63523809 4.7096646  4.66030494 4.6395216  4.8908